In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
page_list = ['https://hesperfox.com/collections/pyjama-shirts', 
            'https://hesperfox.com/collections/pyjama-bottoms',
            'https://hesperfox.com/collections/slips-and-nightdresses',
            'https://hesperfox.com/collections/camisoles-and-shorts',
            'https://hesperfox.com/collections/lingerie',
            'https://hesperfox.com/collections/robes',
            'https://hesperfox.com/collections/sleep-masks']

In [3]:
soup = BeautifulSoup(requests.get(page_list[0]).text)

In [23]:
soup.find_all('div', {'class': 'h4 grid-view-item__title'})[0].text

'Dietrich Silk Pyjama Shirt'

In [46]:
soup.find_all('div', {'class': 'grid-view-item__meta-wrapper'})[9].\
    find('span', {'class': 'product-price__price'}).text

'£165'

In [47]:
soup.find_all('div', {'class': 'grid-view-item__meta-wrapper'})[1]

<div class="grid-view-item__meta-wrapper">
<div class="h4 grid-view-item__title">Pepper Blue Linen Pyjama Shirt</div>
<div class="grid-view-item__meta">
<!-- snippet/product-price.liquid -->
<span class="visually-hidden">Regular price</span>
<s class="product-price__price">£85</s>
<span class="product-price__price product-price__sale">
      £51
      <span class="product-price__sale-label">Sale</span>
</span>
</div>
</div>

In [22]:
soup.find_all('div', {'class': 'grid-view-item__image-wrapper js'})[0].img.get('src')

<img alt="Dietrich Silk Pyjama Shirt" class="grid-view-item__image lazyload" data-aspectratio="0.6666666666666666" data-sizes="auto" data-src="//cdn.shopify.com/s/files/1/1007/3378/products/Dietrich_PJs_shirt_green_{width}x.jpg?v=1568339956" data-widths="[180, 360, 540, 720, 900, 1080, 1296, 1512, 1728, 2048]" id="ProductCardImage-collection-shop-template-1901289308231" src="//cdn.shopify.com/s/files/1/1007/3378/products/Dietrich_PJs_shirt_green_300x300.jpg?v=1568339956"/>

In [108]:
def parse_one_page(html):
    base_url = 'https://hesperfox.com'
    soup = BeautifulSoup(html)
    info = len(soup.find_all('div', {'class': 'grid-view-item__meta-wrapper'}))
    for i in range(info):
        brand.append('Hesper Fox')
        item.append(soup.find_all('div', {'class': 'h4 grid-view-item__title'})[i].text)
        price.append(re.findall(r'\D\d+', soup.find_all('div', {'class': 'grid-view-item__meta-wrapper'})[i].\
                find('span', {'class': 'product-price__price'}).text)[0])
        if soup.find_all('div', {'class': 'grid-view-item__meta-wrapper'})[i].\
            find('span',{'class': 'product-price__sale-label'}) != None:
            deal_price.append(re.findall(r'\D\d+', soup.find_all('div', {'class': 'grid-view-item__meta-wrapper'})[i].\
                            find('span',{'class': 'product-price__price product-price__sale'}).text)[0])
        else:
            deal_price.append(None)
        img_url.append(soup.find_all('div', {'class': 'grid-view-item__image-wrapper js'})[i].img.get('src'))
        product_url.append(base_url + \
                      soup.find_all('a', {'class': 'grid-view-item__link grid-view-item__image-container'})[i]['href'])
    del soup

In [27]:
soup.find_all('a', {'class': 'grid-view-item__link grid-view-item__image-container'})[0]['href']

'/collections/pyjama-shirts/products/dietrich-silk-pyjama-shirt-1'

In [129]:
brand = []
item = []
price = []
deal_price = []
img_url = []
product_url = []


#parse_one_page(requests.get(page_list[1]).text)

In [96]:
from selenium import webdriver
import time

In [94]:
driver = webdriver.Chrome()

In [130]:
for i,j in enumerate(page_list):
    driver.get(j)
    time.sleep(1)

    for i in range(15):
        js='window.scrollBy(0,500)'
        driver.execute_script(js)
        time.sleep(0.5)

    page_source = driver.page_source
    parse_one_page(page_source)
    del page_source

In [132]:
pd.DataFrame(list(zip(brand, item, price, deal_price, img_url, product_url)),
            columns=['brand', 'item', 'price', 'deal_price', 'img_url', 'product_url'])

,brand,item,price,deal_price,img_url,product_url
0,Hesper Fox,Dietrich Silk Pyjama Shirt,£122,£122,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/pyjama-shirt...
1,Hesper Fox,Pepper Blue Linen Pyjama Shirt,£51,£51,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/pyjama-shirt...
2,Hesper Fox,Dietrich Pale Pink Silk Pyjama Shirt,£175,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/pyjama-shirt...
3,Hesper Fox,Pepper Pink Linen Pyjama Shirt,£85,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/pyjama-shirt...
4,Hesper Fox,Lulu Black Short-sleeved Silk Sleep Shirt,£150,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/pyjama-shirt...
...,...,...,...,...,...,...
95,Hesper Fox,Kitsune Monochrome Silk Sleep Mask,£50,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/sleep-masks/...
96,Hesper Fox,Kitsune Emerald Stripe Silk Sleep Mask,£50,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/sleep-masks/...
97,Hesper Fox,Tonight's the Night Silk Sleep Mask,£45,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/sleep-masks/...
98,Hesper Fox,Because the Night Silk Sleep Mask,£45,None,//cdn.shopify.com/s/files/1/1007/3378/products...,https://hesperfox.com/collections/sleep-masks/...


In [133]:
counter = 1

for i in img_url:
    with open("HesperFox" + str(counter) + ".jpg", "wb") as f:
        r = requests.get('https:' + i)
        f.write(r.content)
    counter += 1

In [131]:
len(item)

100